# Apercal tutorial 2: Calibration and imaging of several APERTIF element beams

After we have successfully calibrated the central element beam of an APERTIF observation we now also want to calibrate and image several beams. For this tutorial we take a dataset from our reference field, the Lockman Hole. Let's start the pipeline again as we did in the last tutorial. Remember to change all the directory paths to your own ones.

In [ ]:
import os;
myusername = os.environ['USER']
if not ('PYTHONPATH' in os.environ and
        '/home/' + myusername + '/apercal' in os.environ['PYTHONPATH']):
      print("WARNING: your apercal directory should be in the $PYTHONPATH")

%config IPCompleter.greedy=True
%matplotlib notebook
import libs.lib as lib
lib.setup_logger('info', logfile='/home/{}/apercal/ipython-notebooks/tutorials/logs/2.log'.format(myusername))

import apercal

Load the config file for the tutorial and look at it.

In [ ]:
prepare = apercal.prepare('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
prepare.show(showall=True)

We have to set `prepare_obsmode = multi_element` since these observations were carried out with the full bandwidth in multi-element mode. The prepare step will copy all 37 beams. You might want to stop the step after the third or fourth copied target dataset. The full processing of all beams would need around a day. The pipeline will still work with fewer copied beams. <br>
Just make sure that all datasets in your directory are correct and not corrupted (most likely you will have to remove the last copied dataset manually since it wasn't copied completely).

In [ ]:
prepare.go()

As before we have to execute the individual preflag substeps manually.

In [ ]:
preflag = apercal.preflag('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))

nchannel = 11008

a = range(0, nchannel, 64) # the subband edges
b = range(1, nchannel, 64)
c = range(63, nchannel, 64)
d = range(16, nchannel, 64) # the two ghosts
e = range(48, nchannel, 64)
l = a + b + c + d + e
preflag.preflag_manualflag_channel = ';'.join(str(ch) for ch in l)

preflag.show()

We want to flag auto-correlations and shadowed antennas. The preflag step automatically recognises the other beams and flags them as well using the same pattern.

In [ ]:
preflag.manualflag()

First derive the bandpass for the flagging. This is automatically applied to all target element beams.

In [ ]:
preflag.aoflagger_bandpass()

Now start aoflagger to flag all the datasets automatically.

In [ ]:
preflag.aoflagger_flag()

Now convert the data to MIRIAD format. Also the CONVERT step takes care of all the different element beams automatically.

In [ ]:
convert = apercal.convert('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
convert.show()
convert.go()

We will do the cross calibration now. The pipeline automatically copies the calibrator solutions over to all element beams (`crosscal_transfer_to_target = True`). Even though the cable delay is in place, we still need to solve for delays due to the 5ns sampling (`crosscal_delay = True`).

In [ ]:
ccal = apercal.ccal('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
ccal.show()
ccal.go()

First we reduce all beams with the same standard paramaters.

In [ ]:
scal = apercal.scal('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
scal.show()

You can easily make a list of the available beams for your dataset.

In [ ]:
import glob
from subs import managefiles

managefiles.director(scal,'ch', scal.basedir)
beams = glob.glob('[0-9][0-9]')
print(beams)

Now iterate over the list of beams with the SELFCAL module and execute the self calibration.

In [ ]:
for beam in beams:
    scal.beam = beam
    scal.go()

You might want to change the paramaters and redo the calibration for individual beams.

First change the beam directory for this module. Then use `scal.reset()` to remove the files and calibration for this beam.

In [ ]:
import subs

scal.beam = '01'
scal.reset()

Then change the parameter you want to change temporarily, for example `scal.parametric = False`.

In [ ]:
scal.parametric = False

And then start the selfcal for this beam.

In [ ]:
scal.go()

If you are satisfied with the self-calibration of all your beams, you can export the data to UVFITS format. Since we want the gains applied to the original resolution of the data, we need to run the LINE module.

In [ ]:
line = apercal.line('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
line.show()

We do not want to subtract the continuum `line_subtract = False` nor do any imaging of the line channels `line_image = False`, but we need to transfer the gains from the continuum `line_transfergains = True` and split the data with the same chunkbandwidth `line_splitdata_chunkbandwidth = 0.02` as we set it for the self-calibration. In addition we want the original channel resolution `line_splitdata_channelbandwidth = 1.2e-05`.

As we did for the self-calibration, we want to run this step on all beams.

In [ ]:
import glob
from subs import managefiles

managefiles.director(scal,'ch', scal.basedir)
beams = glob.glob('[0-9][0-9]')

for beam in beams:
    line.beam = beam
    line.go()

Now use the TRANSFER module to write the data out into UVFITS format (`transfer_convert_lineuv2uvfits = True`). All gain tables and flags are applied and all frequency chunks of each beam are combined into one file, so that only one UVFITS-file per beam is written. Multiple beams are automatically handled.

In [ ]:
transfer = apercal.transfer('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
transfer.show()

In [ ]:
transfer.go()

Now we want to create the final continuum images using the same parameters as we used during selfcal. The continuum image just adds another cycle of imaging after the last self-calibration cycle.

In [ ]:
continuum = apercal.continuum('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
continuum.show()

Do the continuum imaging for all beams by iterating with the same loop as we did in the SELFCAL module.

In [ ]:
import glob
from subs import managefiles

managefiles.director(continuum,'ch', continuum.basedir)

beams = glob.glob('[0-9][0-9]')

for beam in beams:
    continuum.beam = beam
    continuum.go()

You can redo the continuum imaging for individual beams in the same way as for the selfcal module above. This holds for all modules where you use a `glob` command to create a list of beams.

In the end we want to mosaic all images together using the MOSAIC module.

In [ ]:
mosaic = apercal.mosaic('/home/{}/apercal/ipython-notebooks/tutorials/cfg/2.cfg'.format(myusername))
mosaic.show()

This tutorial only does a mosaic of the final continuum images (parameter `mosaic_continuum = True`) and mosaics of the individual continuum frequency chunks (parameter `mosaic_continuum_chunks = True`). Mosaics of the line cubes (parameter `mosaic_line`) and Q- and U- images (paramater `mosaic_polarisation`) can also be produced in a later release.

In [ ]:
mosaic.go()